In [ ]:
# -*- coding: utf-8 -*-
"""EXPERIMENT_15_TASK_11_DROPOUT_0.9
Automatically generated by Colaboratory.
Original file is located at https://colab.research.google.com/drive/1t2ym5S65ZUurXSAfPEIbZvVTTnKCluY
Z """
# %matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt import matplotlib.ticker as ticker import urllib
import sys
import os
import hashlib
import re
import itertools
from tqdm import tqdm
#connecting the Google drive
from google.colab import drive drive.mount('/content/drive')
train_set_post_file = '/content/drive/My Drive/GloVe/en-10k/qa11_basic- coreference_train.txt'
test_set_post_file = '/content/drive/My Drive/GloVe/en-10k/qa11_basic- coreference_test.txt'
glove_vectors_file = '/content/drive/My Drive/GloVe/glove.6B.50d.txt'
# Prepare glove_wordmap
glove_wordmap = {}
with open(glove_vectors_file, "r", encoding="utf8") as glove:
for line in glove:
name, vector = tuple(line.split(" ", 1)) glove_wordmap[name] = np.fromstring(vector, sep=" ")
wvecs = []
for item in glove_wordmap.items():
wvecs.append(item[1]) s = np.vstack(wvecs)
v = np.var(s,0)
m = np.mean(s,0)
RS = np.random.RandomState()
def fill_unk(unk):
global glove_wordmap
glove_wordmap[unk] = RS.multivariate_normal(m,np.diag(v)) return glove_wordmap[unk]
def sentence2sequence(sentence):
#This is where the tokenization of words happens. The sentences are stripped
63
#and splitted on spaces and any brackets, commas and hypens are stripped.
#This function will be used inside of the contextualise functinon.
tokens = sentence.strip('"(),-').lower().split(" ") rows = []
words = []
for token in tokens: i = len(token)
while len(token) > 0:
word = token[:i]
if word in glove_wordmap:
rows.append(glove_wordmap[word]) #appending happens with the glove vector #This is where word embeddings happen
  words.append(word) token = token[i:] i = len(token) continue
else:
i = i-1
if i == 0:
rows.append(fill_unk(token)) words.append(token)
break
return np.array(rows), words
def contextualize(set_file):
#This function handles the tokenization part
#The split function is used to tokenize at spaces and tab is used to #identify answers and the supporting factors
#but we dont use the supporting factors
data = []
context = []
with open(set_file, "r", encoding="utf8") as train:
for line in train:
l, ine = tuple(line.split(" ", 1))
# Split the line numbers from the sentences they refer to. if l is "1":
# New contexts always start with 1,
# so this is a signal to reset the context. context = []
if "\t" in ine:
 # Tabs are the separator between questions and answers, # and are not present in context statements.
question, answer, support = tuple(ine.split("\t")) data.append((tuple(zip(*context))+
sentence2sequence(question)+ sentence2sequence(answer)+
([int(s) for s in support.split()],)))
# Multiple questions may refer to the same context, so we don't reset it.
else:
# Context sentence. context.append(sentence2sequence(ine[:-1]))
return data
train_data = contextualize(train_set_post_file) test_data = contextualize(test_set_post_file)
final_train_data = []
64

def finalize(data):
#the contextualise function concatonates the sentences #finalize function is where the sentence ends are indexed.
final_data = [] for cqas in data:
contextvs, contextws, qvs, qws, avs, aws, spt = cqas
lengths = itertools.accumulate(len(cvec) for cvec in contextvs) context_vec = np.concatenate(contextvs)
context_words = sum(contextws,[])
# Location markers for sentence ends/beginnings
sentence_ends = np.array(list(lengths))
final_data.append((context_vec, sentence_ends, qvs, spt, context_words, cqas, avs, aws))
  return np.array(final_data)
#execution of the finalise function
final_train_data = finalize(train_data) final_test_data = finalize(test_data)
tf.reset_default_graph() # Hyperparameters
# The number of dimensions used to store data passed between recurrent layers in the network.
# the layer size.
recurrent_cell_size = 128
# The number of dimensions in our word vectorizations. #we use the glove 50D, but we have 100D and 300 aswell. D = 50
# How quickly the network learns.
learning_rate = 0.005 #regularization
input_p, output_p = 0.9, 0.5
# How many questions we train on at a time. #increasing this takes up considerable about of GPU
batch_size = 128
 # Number of episodic memory passes
passes = 4
# The strength of our regularization.
weight_decay = 0.00000001
# How many questions are trained by the network. This decides the epochs
training_iterations_count = 400000
# How many iterations of training occur before each validation check. # set it to 79 to check validation accuracy at every epoch. display_step = 79
# Input Module
# Context: tensor that contains all the context information.
65

context = tf.placeholder(tf.float32, [None, None, D], "context") context_placeholder = context # I use context as a variable name later on
# input_sentence_endings: tensor that contains the locations of the ends of sentences.
input_sentence_endings = tf.placeholder(tf.int32, [None, None, 2], "sentence")
# recurrent_cell_size: the number of hidden units in recurrent layers.
input_gru = tf.contrib.rnn.GRUCell(recurrent_cell_size)
#he dropoutwrapper is used to put the dropout to the input module for rwgularization
gru_drop = tf.contrib.rnn.DropoutWrapper(input_gru, input_p, output_p)
  input_module_outputs, _ = tf.nn.dynamic_rnn(gru_drop, context, dtype=tf.float32, scope = "input_module")
# cs: the facts gathered from the context.
cs = tf.gather_nd(input_module_outputs, input_sentence_endings) s = input_module_outputs
# Question Module
# query: tensor that contains all of the questions.
query = tf.placeholder(tf.float32, [None, None, D], "query")
# input_query_lengths: tensor that contains question length information. # input_query_lengths[:,1] has the actual lengths; input_query_lengths[:,0] is a simple range()
input_query_lengths = tf.placeholder(tf.int32, [None, 2], "query_lengths")
question_module_outputs, _ = tf.nn.dynamic_rnn(gru_drop, query, dtype=tf.float32,
scope =
tf.VariableScope(True, "input_module")) # q: the question states.
q = tf.gather_nd(question_module_outputs, input_query_lengths) # Episodic Memory
 size = tf.stack([tf.constant(1),tf.shape(cs)[1], tf.constant(1)]) re_q = tf.tile(tf.reshape(q,[-1,1,recurrent_cell_size]),size)
output_size = 1
# Weights and biases
attend_init = tf.random_normal_initializer(stddev=0.1)
w_1 = tf.get_variable("attend_w1", [1,recurrent_cell_size*7, recurrent_cell_size],
tf.float32, initializer = attend_init)
w_2 = tf.get_variable("attend_w2", [1,recurrent_cell_size, output_size],
tf.float32, initializer = attend_init)
b_1 = tf.get_variable("attend_b1", [1, recurrent_cell_size], tf.float32, initializer = attend_init)
66

b_2 = tf.get_variable("attend_b2", [1, output_size], tf.float32, initializer = attend_init)
# Regulate all the weights and biases
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, tf.nn.l2_loss(w_1)) tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, tf.nn.l2_loss(b_1)) tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, tf.nn.l2_loss(w_2)) tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, tf.nn.l2_loss(b_2))
#Attention Mechanism
  def attention(c, mem, existing_facts):
# c: tensor that contains all the facts from the contexts.
# mem: tensor that contains the current memory.
with tf.variable_scope("attending") as scope:
# in this function the context and the current memory is
concatonated.
attending = tf.concat([c, mem, re_q, c * re_q, c * mem, (c- re_q)**2, (c-mem)**2], 2)
# m1: First layer of multiplied weights for the feed-forward network.
m1 = tf.matmul(attending * existing_facts, tf.tile(w_1,
tf.stack([tf.shape(attending)[0],1,1]))) * existing_facts
# bias_1: A masked version of the first feed-forward layer's
bias
# over only existing facts.
bias_1 = b_1 * existing_facts
tnhan = tf.nn.relu(m1 + bias_1)
# m2: Second layer of multiplied weights for the feed-forward network.
m2 = tf.matmul(tnhan, tf.tile(w_2, tf.stack([tf.shape(attending)[0],1,1])))
 bias.
# bias_2: A masked version of the second feed-forward layer's
bias_2 = b_2 * existing_facts
norm_m2 = tf.nn.l2_normalize(m2 + bias_2, -1)
softmax_idx = tf.where(tf.not_equal(norm_m2, 0))[:,:-1] softmax_gather = tf.gather_nd(norm_m2[...,0], softmax_idx) softmax_shape = tf.shape(norm_m2, out_type=tf.int64)[:-1] softmaxable = tf.SparseTensor(softmax_idx, softmax_gather,
softmax_shape) return
tf.expand_dims(tf.sparse_tensor_to_dense(tf.sparse_softmax(softmaxable)) ,-1)
67

# facts_0s: tensor whose values are 1 if the corresponding fact exists and 0 if not.
facts_0s = tf.cast(tf.count_nonzero(input_sentence_endings[:,:,-1:],- 1,keep_dims=True),tf.float32)
with tf.variable_scope("Episodes") as scope:
attention_gru = tf.contrib.rnn.GRUCell(recurrent_cell_size)
# memory: A list of all tensors that are the (current or past) memory state
    # of the attention mechanism.
memory = [q]
# attends: A list of all tensors that represent what the network
  attends to.
attends = []
for a in range(passes):
        # attention mask
attend_to = attention(cs, tf.tile(tf.reshape(memory[-1],[- 1,1,recurrent_cell_size]),size),
facts_0s)
# Inverse attention mask, for what's retained in the state.
retain = 1-attend_to
# GRU pass over the facts, according to the attention mask.
while_valid_index = (lambda state, index: index < tf.shape(cs)[1])
update_state = (lambda state, index: (attend_to[:,index,:] *
attention_gru(cs[:,index,:], state)[0] +
retain[:,index,:] *
state))
# start loop with most recent memory and at the first index memory.append(tuple(tf.while_loop(while_valid_index,
(lambda state, index: (update_state(state,index),index+1)),
loop_vars = [memory[-1], 0]))[0]) attends.append(attend_to)
scope.reuse_variables()
 # Answer Module
# a0: Final memory state
a0 = tf.concat([memory[-1], q], -1)
fc_init = tf.random_normal_initializer(stddev=0.1) with tf.variable_scope("answer"):
w_answer = tf.get_variable("weight", [recurrent_cell_size*2, D], tf.float32, initializer = fc_init)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, tf.nn.l2_loss(w_answer))
logit = tf.expand_dims(tf.matmul(a0, w_answer),1)
68

with tf.variable_scope("ending"):
all_ends = tf.reshape(input_sentence_endings, [-1,2]) range_ends = tf.range(tf.shape(all_ends)[0])
ends_indices = tf.stack([all_ends[:,0],range_ends], axis=1) ind = tf.reduce_max(tf.scatter_nd(ends_indices, all_ends[:,1],
[tf.shape(q)[0],
tf.shape(all_ends)[0]]),
axis=-1)
range_ind = tf.range(tf.shape(ind)[0])
mask_ends = tf.cast(tf.scatter_nd(tf.stack([ind, range_ind], axis=1),
 [tf.reduce_max(ind)+1, tf.shape(ind)[0]]), bool)
tf.ones_like(range_ind),
 mask = tf.scan(tf.logical_xor,mask_ends, tf.ones_like(range_ind, dtype=bool))
logits = - tf.reduce_sum(tf.square(context*tf.transpose(tf.expand_dims(
tf.cast(mask, tf.float32),-1),[1,0,2]) - logit),
axis=-1)
# Training
gold_standard = tf.placeholder(tf.float32, [None, 1, D], "answer") with tf.variable_scope('accuracy'):
eq = tf.equal(context, gold_standard)
corrbool = tf.reduce_all(eq,-1)
logloc = tf.reduce_max(logits, -1, keep_dims = True)
locs = tf.equal(logits, logloc)
correctsbool = tf.reduce_any(tf.logical_and(locs, corrbool), -1)
corrects = tf.where(correctsbool, tf.ones_like(correctsbool, dtype=tf.float32),
tf.zeros_like(correctsbool,dtype=tf.float32))
corr = tf.where(corrbool, tf.ones_like(corrbool, dtype=tf.float32), tf.zeros_like(corrbool,dtype=tf.float32))
with tf.variable_scope("loss"):
 # Use sigmoid cross entropy as the base loss.
# Could try softmax cross entropy and see also.
loss = tf.nn.sigmoid_cross_entropy_with_logits(logits =
tf.nn.l2_normalize(logits,-1),labels = corr)
# Add regularization losses, weighted by weight_decay.
total_loss = tf.reduce_mean(loss) + weight_decay * tf.add_n(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
# Adam optimizor is chosen, which generally works well for most cases.
optimizer = tf.train.AdamOptimizer(learning_rate)
# Once we have an optimizer, we ask it to minimize the loss
opt_op = optimizer.minimize(total_loss)
# Initialize variables
init = tf.global_variables_initializer()
69

 # Launch the TensorFlow session
sess = tf.Session() sess.run(init)
def prep_batch(batch_data, more_data = False):
context_vec, sentence_ends, questionvs, spt, context_words, cqas, answervs, _ = zip(*batch_data)
ends = list(sentence_ends)
maxend = max(map(len, ends))
aends = np.zeros((len(ends), maxend)) for index, i in enumerate(ends):
for indexj, x in enumerate(i): aends[index, indexj] = x-1
 new_ends = np.zeros(aends.shape+(2,))
for index, x in np.ndenumerate(aends): new_ends[index+(0,)] = index[0] new_ends[index+(1,)] = x
contexts = list(context_vec)
max_context_length = max([len(x) for x in contexts]) contextsize = list(np.array(contexts[0]).shape) contextsize[0] = max_context_length
final_contexts = np.zeros([len(contexts)]+contextsize)
contexts = [np.array(x) for x in contexts] for i, context in enumerate(contexts):
final_contexts[i,0:len(context),:] = context max_query_length = max(len(x) for x in questionvs) querysize = list(np.array(questionvs[0]).shape) querysize[:1] = [len(questionvs),max_query_length] queries = np.zeros(querysize)
querylengths = np.array(list(zip(range(len(questionvs)),[len(q)-1 for q in questionvs])))
questions = [np.array(q) for q in questionvs] for i, question in enumerate(questions):
queries[i,0:len(question),:] = question
data = {context_placeholder: final_contexts, input_sentence_endings:
new_ends,
query:queries, input_query_lengths:querylengths, gold_standard: answervs}
return (data, context_words, cqas) if more_data else data
 # Prepare validation set
batch = np.random.randint(final_test_data.shape[0], size=batch_size*10) batch_data = final_test_data[batch]
validation_set, val_context_words, val_cqas = prep_batch(batch_data, True)
def train(iterations, batch_size):
training_iterations = range(0, iterations, batch_size) training_iterations = tqdm(training_iterations)
accuracy_list = [] loss_list = [] epoc_list = []
k=1
wordz = []
70

for j in training_iterations:
 batch = np.random.randint(final_train_data.shape[0], size=batch_size)
batch_data = final_train_data[batch] sess.run([opt_op], feed_dict=prep_batch(batch_data))
if (j/batch_size) % display_step == 0:
            # Calculate batch accuracy
acc, ccs, tmp_loss, log, con, cor, loc = sess.run([corrects, cs, total_loss, logit, context_placeholder,corr, locs], feed_dict=validation_set)
print("Iter " + str(j/batch_size) + ", Minibatch Loss= ",
 tmp_loss, "Accuracy= ", np.mean(acc)) accuracy_list.append(np.mean(acc))
loss_list.append(tmp_loss) epoc_list.append(k)
k=k+1
import matplotlib.pyplot as plt plt.style.use('seaborn-white') plt.plot(epoc_list,accuracy_list, color='steelblue') plt.xlabel('Epoch')
plt.ylabel('Accuracy') plt.savefig('accuracy.png', dpi=300) plt.show()
plt.plot(epoc_list,loss_list, color='steelblue') plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.savefig('loss.png', dpi=300)
plt.show() train(training_iterations_count, batch_size)
# Final testing accuracy
print("Test Accuracy : ") print(np.mean(sess.run([corrects], feed_dict= prep_batch(final_test_data))[0]))
ancr = sess.run([corrbool,locs, total_loss, logits, facts_0s, w_1]+attends+[query, cs,
 question_module_outputs],feed_dict=validation_set)
a = ancr[0]
n = ancr[1]
cr = ancr[2]
attenders = np.array(ancr[6:-3]) faq = np.sum(ancr[4], axis=(-1,-2))
limit = 5
for question in range(min(limit, batch_size)):
plt.yticks(range(passes,0,-1))
plt.ylabel("Episode")
plt.xlabel("Question "+str(question+1))
pltdata = attenders[:,question,:int(faq[question]),0] pltdata = (pltdata - pltdata.mean()) / ((pltdata.max() -
pltdata.min() + 0.001)) * 256
plt.pcolor(pltdata, cmap=plt.cm.Blues, alpha=0.7)
71

plt.show()
# this get the predicted words location within the context.
indices = np.argmax(n,axis=1)
# Locations of actual answers within contexts
indicesc = np.argmax(a,axis=1)
for i,e,cw, cqa in list(zip(indices, indicesc, val_context_words, val_cqas))[:limit]:
ccc = " ".join(cw)
print("TEXT: ",ccc)
print ("QUESTION: ", " ".join(cqa[3]))
print ("RESPONSE: ", cw[i], ["Correct", "Incorrect"][i!=e])
print("EXPECTED: ", cw[e]) print()
from google.colab import files files.download('accuracy.png') files.download('loss.png')
sess.close()